In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd ./drive/MyDrive/github

/content/drive/MyDrive/github


In [3]:
!git clone https://github.com/CodedotAl/code_clippy_lm_dataformat.git

Cloning into 'code_clippy_lm_dataformat'...
remote: Enumerating objects: 279, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 279 (delta 54), reused 47 (delta 47), pack-reused 223
Receiving objects: 100% (279/279), 87.17 KiB | 2.03 MiB/s, done.
Resolving deltas: 100% (146/146), done.


# Prepare Environment

In [8]:
%cd code_clippy_lm_dataformat

/content/drive/MyDrive/github/code_clippy_lm_dataformat


In [10]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/github/code_clippy_lm_dataformat
     |████████████████████████████████| 2.5 MB 5.3 MB/s 
  Running setup.py develop for lm-dataformat


In [17]:
!pip uninstall python-magic

Found existing installation: python-magic 0.4.27
Uninstalling python-magic-0.4.27:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/magic/*
    /usr/local/lib/python3.7/dist-packages/python_magic-0.4.27.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled python-magic-0.4.27
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


In [22]:
!wget https://files.pythonhosted.org/packages/6c/73/9f872cb81fc5c3bb48f7227872c28975f998f3e7c2b1c16e95e6432bbb90/python_magic-0.4.27-py2.py3-none-any.whl

--2022-08-16 17:34:11--  https://files.pythonhosted.org/packages/6c/73/9f872cb81fc5c3bb48f7227872c28975f998f3e7c2b1c16e95e6432bbb90/python_magic-0.4.27-py2.py3-none-any.whl
Resolving files.pythonhosted.org (files.pythonhosted.org)... 151.101.1.63, 151.101.65.63, 151.101.129.63, ...
Connecting to files.pythonhosted.org (files.pythonhosted.org)|151.101.1.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13840 (14K) [application/octet-stream]
Saving to: ‘python_magic-0.4.27-py2.py3-none-any.whl’

python_magic-0.4.27 100%[===================>]  13.52K  --.-KB/s    in 0.006s  

2022-08-16 17:34:11 (2.23 MB/s) - ‘python_magic-0.4.27-py2.py3-none-any.whl’ saved [13840/13840]



In [25]:
!wget https://files.pythonhosted.org/packages/43/79/3bf793efcf94e730bf324b726b3ae551f9438869bb8995600784bdb17693/python-libmagic-0.4.0.tar.gz

--2022-08-16 17:36:35--  https://files.pythonhosted.org/packages/43/79/3bf793efcf94e730bf324b726b3ae551f9438869bb8995600784bdb17693/python-libmagic-0.4.0.tar.gz
Resolving files.pythonhosted.org (files.pythonhosted.org)... 151.101.1.63, 151.101.65.63, 151.101.129.63, ...
Connecting to files.pythonhosted.org (files.pythonhosted.org)|151.101.1.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3592 (3.5K) [application/octet-stream]
Saving to: ‘python-libmagic-0.4.0.tar.gz’

python-libmagic-0.4 100%[===================>]   3.51K  --.-KB/s    in 0.002s  

2022-08-16 17:36:35 (1.47 MB/s) - ‘python-libmagic-0.4.0.tar.gz’ saved [3592/3592]



In [23]:
!pip install python_magic-0.4.27-py2.py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./python_magic-0.4.27-py2.py3-none-any.whl


In [32]:
!sudo apt-get install libmagic-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-dev libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 332 kB of archives.
After this operation, 5,552 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-dev amd64 1:5.32-2ubuntu0.4 [79.7 kB]
Fetched 332 kB in 1s (354 kB/s)
debconf: unable to initialize frontend: Dialo

In [33]:
import lm_dataformat as lmd

import hashlib
import shutil

In [34]:
def test_jsonl():
    archive = lmd.Archive('test_dir')
    blns = open('test/blns.txt').read()
    archive.add_data(blns)
    archive.add_data('testing 123', meta={'testing': 123})
    archive.add_data(blns, meta={'testing2': 456, 'testing': ['a','b']})
    archive.add_data('testing 123456789')
    archive.commit()

    reader = lmd.Reader('test_dir')

    data = list(reader.stream_data(get_meta=True))

    assert data[0] == (blns, {})
    assert data[1] == ('testing 123', {'testing': 123})
    assert data[2] == (blns, {'testing2': 456, 'testing': ['a','b']})
    assert data[3] == ('testing 123456789', {})
    shutil.rmtree('test_dir')

In [38]:
archive = lmd.Archive('test_dir')
blns = open('./test/blns.txt').read()
archive.add_data(blns)
archive.add_data('testing 123', meta={'testing': 123})
archive.add_data(blns, meta={'testing2': 456, 'testing': ['a','b']})
archive.add_data('testing 123456789')
archive.commit()

reader = lmd.Reader('./test_dir')

data = list(reader.stream_data(get_meta=True))

TypeError: ignored

# To READ

In [ ]:
rdr = Reader('input_dir_or_file')

for doc in rdr.stream_data():
  # do something with the document